In [ ]:
import wandb
import pandas as pd
import json
from dataclasses import dataclass
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from pathlib import Path
from collections import defaultdict

import scienceplots
# plt.style.use(['science', 'ieee'])

In [ ]:
api = wandb.Api(timeout=19)
entity, project = "mtm_team", "rep_exorl_experiments"

runs = api.runs(entity + "/" + project)
print(f"Found {len(runs)} runs in {entity}/{project}")

In [ ]:
r = runs[30]
hist = r.history()
hist.keys()

In [ ]:
hist["eval/episode_reward"]

In [ ]:
# Filter runs
# allowed_notes = ["rss sweep ablation (control_timestep)"]

# pig_runs = []
# for run in runs:
#     if run.config["notes"] not in allowed_notes or run.state not in ["finished", "running"]:
#         continue
#     pig_runs.append(run)
# print(f"Found {len(pig_runs)} runs related to this ablation.")

In [ ]:
# Group runs with the same control_timestep together.


def g(r):
    return r.config["task"]

task_map = defaultdict(list)
for run in runs:
    task_map[g(run)].append(run)

In [ ]:
# Grab the eval sub-runs for each seed.

def g(r):
    if r.config["agent"]["name"] == "td3":
        return "td3"
    else:
        if r.config["agent"]["end_to_end"]:
            return "td3_mtm_e2e"
        else:
            return  "td3_mtm"
        

plot_map = defaultdict(lambda: defaultdict(list))
for k, runs in task_map.items():
    for run in runs:
        plot_map[k][g(run)].append(run.history(keys=["eval/episode_reward"]))
        print(run.name)

In [ ]:
plot_map["walker_run"]["td3"]

In [ ]:
# time_series = {}
# for env_name, runs in plot_map.items():
#     time_series[env_name] = []
#     for run in runs:
#         time_series[env_name].append(run.history()["eval/episode_reward"])

In [ ]:
np.array(s).shape

In [ ]:
for env in plot_map.keys():
    # Plot the time series, with the mean and standard deviation over the 1M steps.
    fig, ax = plt.subplots(1, 1, figsize=(6, 4), constrained_layout=True)

    
    for ctrl_timestep, series in plot_map[env].items():
        rew_per_step = defaultdict(list)
        for s in series:
            for r in np.array(s):
                rew_per_step[r[0]].append(r[1])
        
        xs = []
        ys = []
        stds = []
        sorted_steps = list(rew_per_step.keys())
        sorted_steps.sort()
        for step in sorted_steps:
            xs.append(step)
            ys.append(np.mean(rew_per_step[step]))
            stds.append(np.std(rew_per_step[step]))
        
        xs = np.array(xs)
        ys = np.array(ys)
        stds = np.array(stds)
        
        # Make x-axis steps in units of 1M.
        ax.plot(xs, ys, label=f"ctrl_timestep={ctrl_timestep}", linewidth=0.5)

        ax.fill_between(xs, ys - stds, ys + stds, alpha=0.5)

    ax.legend()
    ax.set_xlabel("Timestep")
    ax.set_ylabel("F1 score")

    # Add vertical light grid with major ticks every 1M steps.
    ax.grid(which="major", axis="x", color="lightgray", linestyle="--")
    # Add horizontal light grid with major ticks every 0.2.
    ax.grid(which="major", axis="y", color="lightgray", linestyle="--")
    plt.show()

In [ ]:
xs = np.array(xs)
ys = np.array(ys)
stds = np.array(stds)

In [ ]:
xs

In [ ]:
ys